# Merge

## import

In [ ]:
import unicodedata

import kanjize
import pandas as pd
from tqdm import tqdm

# setting
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 500)

tqdm.pandas()

## zipcode
https://www.post.japanpost.jp/zipcode/dl/oogaki-zip.html

```
全国地方公共団体コード（JIS X0401、X0402）………　半角数字
（旧）郵便番号（5桁）………………………………………　半角数字
郵便番号（7桁）………………………………………　半角数字
都道府県名　…………　半角カタカナ（コード順に掲載）　（※1）
市区町村名　…………　半角カタカナ（コード順に掲載）　（※1）
町域名　………………　半角カタカナ（五十音順に掲載）　（※1）
都道府県名　…………　漢字（コード順に掲載）　（※1,2）
市区町村名　…………　漢字（コード順に掲載）　（※1,2）
町域名　………………　漢字（五十音順に掲載）　（※1,2）
一町域が二以上の郵便番号で表される場合の表示　（※3）　（「1」は該当、「0」は該当せず）
小字毎に番地が起番されている町域の表示　（※4）　（「1」は該当、「0」は該当せず）
丁目を有する町域の場合の表示　（「1」は該当、「0」は該当せず）
一つの郵便番号で二以上の町域を表す場合の表示　（※5）　（「1」は該当、「0」は該当せず）
更新の表示（※6）（「0」は変更なし、「1」は変更あり、「2」廃止（廃止データのみ使用））
変更理由　（「0」は変更なし、「1」市政・区政・町政・分区・政令指定都市施行、「2」住居表示の実施、「3」区画整理、「4」郵便区調整等、「5」訂正、「6」廃止（廃止データのみ使用））
```

In [ ]:
filename = 'data/KEN_ALL.csv'
names = [
    "全国地方公共団体コード",
    "（旧）郵便番号",
    "郵便番号",
    "都道府県名カタカナ",
    "市区町村名カタカナ",
    "町域名カタカナ",
    "都道府県名",
    "市区町村名",
    "町域名",
    "一町域が二以上の郵便番号で表される場合の表示",
    "小字毎に番地が起番されている町域の表示",
    "丁目を有する町域の場合の表示",
    "一つの郵便番号で二以上の町域を表す場合の表示",
    "更新の表示",
    "変更理由",
]
zipcode_df = pd.read_csv(filename, encoding='shift-jis', names=names)
print(zipcode_df.shape)
zipcode_df.head()

(124750, 15)


,全国地方公共団体コード,（旧）郵便番号,郵便番号,都道府県名カタカナ,市区町村名カタカナ,町域名カタカナ,都道府県名,市区町村名,町域名,一町域が二以上の郵便番号で表される場合の表示,小字毎に番地が起番されている町域の表示,丁目を有する町域の場合の表示,一つの郵便番号で二以上の町域を表す場合の表示,更新の表示,変更理由
0,1101,60,600000,ﾎﾂｶｲﾄﾞｳ,ｻﾂﾎﾟﾛｼﾁﾕｳｵｳｸ,ｲｶﾆｹｲｻｲｶﾞﾅｲﾊﾞｱｲ,北海道,札幌市中央区,以下に掲載がない場合,0,0,0,0,0,0
1,1101,64,640941,ﾎﾂｶｲﾄﾞｳ,ｻﾂﾎﾟﾛｼﾁﾕｳｵｳｸ,ｱｻﾋｶﾞｵｶ,北海道,札幌市中央区,旭ケ丘,0,0,1,0,0,0
2,1101,60,600041,ﾎﾂｶｲﾄﾞｳ,ｻﾂﾎﾟﾛｼﾁﾕｳｵｳｸ,ｵｵﾄﾞｵﾘﾋｶﾞｼ,北海道,札幌市中央区,大通東,0,0,1,0,0,0
3,1101,60,600042,ﾎﾂｶｲﾄﾞｳ,ｻﾂﾎﾟﾛｼﾁﾕｳｵｳｸ,ｵｵﾄﾞｵﾘﾆｼ(1-19ﾁﾖｳﾒ),北海道,札幌市中央区,大通西（１〜１９丁目）,1,0,1,0,0,0
4,1101,64,640820,ﾎﾂｶｲﾄﾞｳ,ｻﾂﾎﾟﾛｼﾁﾕｳｵｳｸ,ｵｵﾄﾞｵﾘﾆｼ(20-28ﾁﾖｳﾒ),北海道,札幌市中央区,大通西（２０〜２８丁目）,1,0,1,0,0,0


In [29]:
# clean
use_cols = ["全国地方公共団体コード", "郵便番号", "都道府県名", "市区町村名", "町域名"]
zipcode_df = zipcode_df[use_cols]

## latitude longitude

In [25]:
filename = 'data/latitude_longitude.csv'
latlong_df = pd.read_csv(filename, encoding='ansi')
print(latlong_df.shape)
latlong_df.head()

(190786, 10)


,都道府県コード,都道府県名,市区町村コード,市区町村名,大字町丁目コード,大字町丁目名,緯度,経度,原典資料コード,大字・字・丁目区分コード
0,1,北海道,1101,札幌市中央区,11010001001,旭ケ丘一丁目,43.042230,141.319722,0,3
1,1,北海道,1101,札幌市中央区,11010001002,旭ケ丘二丁目,43.039768,141.321733,0,3
2,1,北海道,1101,札幌市中央区,11010001003,旭ケ丘三丁目,43.039569,141.319617,0,3
3,1,北海道,1101,札幌市中央区,11010001004,旭ケ丘四丁目,43.038819,141.323040,0,3
4,1,北海道,1101,札幌市中央区,11010001005,旭ケ丘五丁目,43.036547,141.322217,0,3


In [33]:
# cealn
use_cols = ["都道府県名", "市区町村コード", "市区町村名", "大字町丁目名", "緯度", "経度"]
latlong_df = latlong_df[use_cols]

## check

In [34]:
# check
a_df = zipcode_df[zipcode_df["市区町村名"] == '札幌市中央区']
a_df

,全国地方公共団体コード,郵便番号,都道府県名,市区町村名,町域名
0,1101,600000,北海道,札幌市中央区,以下に掲載がない場合
1,1101,640941,北海道,札幌市中央区,旭ケ丘
2,1101,600041,北海道,札幌市中央区,大通東
3,1101,600042,北海道,札幌市中央区,大通西（１〜１９丁目）
4,1101,640820,北海道,札幌市中央区,大通西（２０〜２８丁目）
5,1101,600031,北海道,札幌市中央区,北一条東
6,1101,600001,北海道,札幌市中央区,北一条西（１〜１９丁目）
7,1101,640821,北海道,札幌市中央区,北一条西（２０〜２８丁目）
8,1101,600032,北海道,札幌市中央区,北二条東
9,1101,600002,北海道,札幌市中央区,北二条西（１〜１９丁目）


In [35]:
# check
b_df = latlong_df[latlong_df["市区町村名"] == '札幌市中央区']
b_df

,都道府県名,市区町村コード,市区町村名,大字町丁目名,緯度,経度
0,北海道,1101,札幌市中央区,旭ケ丘一丁目,43.042230,141.319722
1,北海道,1101,札幌市中央区,旭ケ丘二丁目,43.039768,141.321733
2,北海道,1101,札幌市中央区,旭ケ丘三丁目,43.039569,141.319617
3,北海道,1101,札幌市中央区,旭ケ丘四丁目,43.038819,141.323040
4,北海道,1101,札幌市中央区,旭ケ丘五丁目,43.036547,141.322217
...,...,...,...,...,...,...
985,北海道,1101,札幌市中央区,宮の森四条九丁目,43.061685,141.299348
986,北海道,1101,札幌市中央区,宮の森四条十丁目,43.060356,141.298776
987,北海道,1101,札幌市中央区,宮の森四条十一丁目,43.058653,141.295779
988,北海道,1101,札幌市中央区,宮の森四条十二丁目,43.055479,141.294501


- no lat long
札幌市白石区 北郷十条

旭川市 １条通（１〜１７丁目）    

留萌市 留萌原野（１〜１２線） 

In [186]:
latlong_df[latlong_df["市区町村名"] == '留萌市']

,都道府県名,市区町村コード,市区町村名,大字町丁目名,緯度,経度
12528,北海道,1212,留萌市,明元町一丁目,43.945270,141.637368
12529,北海道,1212,留萌市,明元町二丁目,43.944601,141.638695
12530,北海道,1212,留萌市,明元町三丁目,43.944084,141.640559
12531,北海道,1212,留萌市,明元町四丁目,43.943710,141.642020
12532,北海道,1212,留萌市,明元町五丁目,43.943249,141.642890
12533,北海道,1212,留萌市,明元町六丁目,43.942376,141.643908
12534,北海道,1212,留萌市,旭町一丁目,43.936633,141.646846
12535,北海道,1212,留萌市,旭町二丁目,43.936069,141.648194
12536,北海道,1212,留萌市,旭町三丁目,43.934856,141.649596
12537,北海道,1212,留萌市,五十嵐町一丁目,43.937203,141.662605


In [185]:
zipcode_df[zipcode_df["町域名"].str.contains('留萌原野（１〜１２線）')]

,全国地方公共団体コード,郵便番号,都道府県名,市区町村名,町域名
2312,1212,770013,北海道,留萌市,留萌原野（１〜１２線）


In [184]:
latlong_df[latlong_df["大字町丁目名"].str.contains('留萌原野')]

,都道府県名,市区町村コード,市区町村名,大字町丁目名,緯度,経度


## test

In [179]:
# https://neu101.seesaa.net/article/159968583.html
#!/usr/bin/env python
# -*- coding: utf8 -*-
import re

char2int = {
  u'0' :0, u'1' :1, u'2' :2, u'3' :3, u'4' :4,
  u'5' :5, u'6' :6, u'7' :7, u'8' :8, u'9' :9,
  u'０':0, u'１':1, u'２':2, u'３':3, u'４':4,
  u'５':5, u'６':6, u'７':7, u'８':8, u'９':9,
}

numKanji0 = [ u'', u'一', u'二', u'三', u'四', u'五', u'六', u'七', u'八', u'九' ]
numKanji1 = [ u'', u'',   u'二', u'三', u'四', u'五', u'六', u'七', u'八', u'九' ]
numKanji  = [ numKanji0, numKanji1, numKanji1, numKanji1 ]

numPlace1 = [ u'', u'十', u'百', u'千' ]
numPlace4 = [ u'', u'万', u'億', u'兆', u'京', u'垓' ]

def convert_pure_integerstring(match):
  source = match.group()
  numstr = re.sub( u'[,，]', u'', source )
  s = []
  for ch in ((u'0'*((4-len(numstr)%4)&3))+numstr): s.insert(0,char2int[ch])

  list = []
  while len(s):
    temp = u''
    for i in range(4):
      if s[i]: temp = numKanji[i][s[i]] + numPlace1[i] + temp
    list.append(temp)
    s = s[4:]

  if len(list) > len(numPlace4): return source

  result = u''  # (a),(b),(c),(d)
  for i in range(len(list)):  # (a),(b),(c),(d)
    if list[i]:
        if len(list) > 1 and list[i] == u'千': list[i] = u'一' + list[i]  # (d)
        result = list[i] + numPlace4[i] + result

  return result if result else u'零'

def convert_integerstring(string):
  if string == None or string == u'': return u''
  p = re.compile(u'[0-9０-９][0-9０-９,，]*[0-9０-９]|[0-9０-９]')
  return p.sub( convert_pure_integerstring, string )

print(convert_integerstring('１条通'))

一条通


In [ ]:
def add_latlong(r):
    # init
    ll_small_df = latlong_df[latlong_df["市区町村名"] == r["市区町村名"]]
    choiki = r["町域名"]
    print(r["市区町村名"], choiki, ' '*30, end='\r')
    # convert arabia num to kanji num
    choiki = convert_integerstring(choiki)

    if choiki == "以下に掲載がない場合":
        ll_match_df = ll_small_df

    # exactly match or partly match "旭ケ丘"
    # ll_match_df = ll_small_df[ll_small_df["大字町丁目名"] == choiki]
    if not choiki == "以下に掲載がない場合":
        ll_match_df = ll_small_df[ll_small_df["大字町丁目名"].str.contains(choiki)]

    if "（" in choiki:
        chome = choiki.split("（")[1]
        choiki = choiki.split("（")[0]
        # in case of "大通西（１〜１９丁目）"
        if "〜" in chome:
            # get chome
            chome = chome.replace("丁目", "")
            chome = chome.replace("番地", "")
            chome = chome.replace("）", "")
            # print(chome)
            chome_s = chome.split("〜")[0]
            chome_e = chome.split("〜")[1]
            # print(choiki, chome_s, chome_e, ' '*30)
            ll_match_df = ll_small_df[ll_small_df["大字町丁目名"].str.contains(choiki)]
            
            # in case of '留萌市 留萌原野（１〜１２線）'
            if not ll_match_df.empty:
                # ll_match_df = ll_match_df.iloc[chome_s-1:chome_e, :]
                # get index start
                temp_ll_df = ll_match_df[ll_match_df["大字町丁目名"] == choiki+chome_s+"丁目"]
                # index start:
                if temp_ll_df.empty:
                    index_s = ll_match_df.index[0]
                else:
                    index_s = temp_ll_df.index[0]
                # get index end
                temp_ll_df = ll_match_df[ll_match_df["大字町丁目名"] == choiki+chome_e+"丁目"]
                # index end:
                if temp_ll_df.empty:
                    index_e = ll_match_df.index[-1]
                else:
                    index_e = temp_ll_df.index[0]
                # print(choiki, index_s, index_e)
                ll_match_df = ll_match_df.loc[index_s:index_e, :]

        # in case of 南郷通（南）'
        else:
            ll_match_df = ll_small_df[ll_small_df["大字町丁目名"].str.contains(choiki)]

    if ll_match_df.empty:
        # ll_match_df = ll_small_df[ll_small_df["大字町丁目名"].str.contains(choiki)]
        print('no result for ', r["市区町村名"], choiki, ' '*30, end='\r')

    lat = ll_match_df["緯度"].mean()
    lon = ll_match_df["経度"].mean()
    # print(lat, lon)
    # return lat, lon
    r['latitude'] = lat
    r['longitude'] = lon
    return pd.Series(r)

aa_df = a_df.apply(add_latlong, axis=1)
print()
aa_df.head()

札幌市中央区 宮の森四条                                       


,全国地方公共団体コード,郵便番号,都道府県名,市区町村名,町域名,latitude,longitude
0,1101,600000,北海道,札幌市中央区,以下に掲載がない場合,43.053100,141.337145
1,1101,640941,北海道,札幌市中央区,旭ケ丘,43.039101,141.320875
2,1101,600041,北海道,札幌市中央区,大通東,43.063593,141.368414
3,1101,600042,北海道,札幌市中央区,大通西（１〜１９丁目）,43.059134,141.341962
4,1101,640820,北海道,札幌市中央区,大通西（２０〜２８丁目）,43.056572,141.320757


In [164]:
aa_df.isna().sum()

全国地方公共団体コード    0
郵便番号           0
都道府県名          0
市区町村名          0
町域名            0
latitude       0
longitude      0
dtype: int64

## run all

In [194]:
# df = zipcode_df.progress_apply(add_latlong, axis=1)
df = zipcode_df.apply(add_latlong, axis=1)
# df = zipcode_df[zipcode_df["市区町村名"] == "札幌市南区"].apply(add_latlong, axis=1)
print(df.shape)
df.head()

(124750, 7)国                                                                                               瀬棚区本町（５区）                                新井牧場                                小山崎                                山田（寺社原、細尾）                                                               犬山                                大栗安                                塩小路町                                太秦乾町                                曙川東                               


,全国地方公共団体コード,郵便番号,都道府県名,市区町村名,町域名,latitude,longitude
0,1101,600000,北海道,札幌市中央区,以下に掲載がない場合,43.053100,141.337145
1,1101,640941,北海道,札幌市中央区,旭ケ丘,43.039101,141.320875
2,1101,600041,北海道,札幌市中央区,大通東,43.063593,141.368414
3,1101,600042,北海道,札幌市中央区,大通西（１〜１９丁目）,43.059134,141.341962
4,1101,640820,北海道,札幌市中央区,大通西（２０〜２８丁目）,43.056572,141.320757


In [195]:
df.isna().sum()

全国地方公共団体コード        0
郵便番号               0
都道府県名              0
市区町村名              0
町域名                0
latitude       11092
longitude      11092
dtype: int64

## save

In [196]:
filename = 'data/zipcode_latitude_longitude.csv'
df.to_csv(filename, index=False)